In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("karimabdulnabi/fruit-classification10-class")
dataset_path = os.path.join(dataset_path, "MY_data")
# Setup path to data folder
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")
predict_dir = os.path.join(dataset_path, "predict")

train_dir, test_dir


100%|██████████| 29.2M/29.2M [00:00<00:00, 54.3MB/s]

Extracting files...


('/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train',
 '/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/test')

In [4]:
#testing out data transformations on fruit dataset classification
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),transforms.ToTensor()
])

In [5]:
# Creating dataset for fruit classification
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")


Train data:
Dataset ImageFolder
    Number of datapoints: 2301
    Root location: /root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 1025
    Root location: /root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [6]:
train_dataloader = DataLoader(dataset=train_data,batch_size=1,num_workers=1,shuffle=True)

test_dataloader = DataLoader(dataset=test_data,batch_size=1,num_workers=1,shuffle=False)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7feaebf38070>,
 <torch.utils.data.dataloader.DataLoader at 0x7feaebf389a0>)